In [11]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from generator import DataGenerator

MODEL_NAME = "depthwise_model"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1



In [2]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [3]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.SeparableConv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [4]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride

    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

    def get_config(self):
        return {
            "kernel_size": self.ksize,
            "stride": self.stride,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [12]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.SeparableConv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.SeparableConv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            #keras.layers.BatchNormalization(),
            #keras.layers.ReLU(),
        ]
        #self.conv_out = keras.layers.Conv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out')
        self.out = self.call(self.input_layer)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        #scale4 = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        #transposed_logs = keras.layers.Lambda(lambda x: tf.transpose(x,(1,0,2)))(logits)
        logits = keras.layers.Softmax()(logits)
        return logits

In [6]:
model = LPRnet()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
model.build((1,24,94,3))
model.summary()

Model: "lp_rnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_conv1 (SeparableConv2D) (None, 24, 94, 64)        283       
_________________________________________________________________
BN1 (BatchNormalization)     (None, 24, 94, 64)        256       
_________________________________________________________________
RELU1 (ReLU)                 (None, 24, 94, 64)        0         
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 24, 94, 64)        0         
_________________________________________________________________
small_basic_block (small_bas (None, 24, 94, 128)       9600      
_________________________________________________________________
maxpool2d_2 (MaxPooling2D)   (None, 24, 47, 128)       0         
_________________________________________________________________
small_basic_block_1 (small_b (None, 24, 47, 256)       3558

In [8]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset

test_dataset = generate_dataset()

C:\Users\carlos\AppData\Local\Temp/ipykernel_24284/3346240790.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [7]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project=MODEL_NAME, entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

def generate_val_set(num=400):
        val_data_gen, val_labels_gen = gen.generate_images(num)
        val_labels_encoded = []
        val_data_encoded = []
        for image,label in zip(val_data_gen, val_labels_gen):
                val_data_encoded.append(np.expand_dims(image,axis=0))
                val_labels_encoded.append([CHARS_DICT[i] for i in label.split('_')[0]])
        return val_data_encoded,val_labels_encoded
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: clsandoval (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
generate = DataGenerator()
model.fit_generator(generator=generate,validation_data=real_dataset,epochs=10,steps_per_epoch=50)

C:\Users\carlos\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


TypeError: 'NoneType' object is not iterable

In [ ]:
for j in range(1000):
    dataset = generate_dataset(num=32000)
    model.fit(dataset,validation_data=real_dataset,validation_steps=5,epochs=10,steps_per_epoch=50,callbacks=[WandbCallback()])
    print(j,flush=True)
    if j %10 == 0:    
        model.save(MODEL_NAME)

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

model.save(MODEL_NAME)

with open("{}.tflite".format(MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpew6g2xkf\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpew6g2xkf\assets


INFO:tensorflow:Assets written to: depthwise_model\assets


INFO:tensorflow:Assets written to: depthwise_model\assets


In [ ]:
interpreter = tf.lite.Interpreter("keras_lprnet_vanilla.tflite")
import numpy as np
import time 
import glob
import cv2

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
start = time.time()
ctr = 0
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    test_image = test_image.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,),greedy=False)
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    print(text, " "+ label)
print(time.time()-start)
print(ctr)

In [19]:
model = keras.models.load_model(
    MODEL_NAME, custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)

In [ ]:
import glob
import cv2
real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
ctr = 0
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    print(text," "+ label)
print(ctr)
print(len(real_images))

In [8]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    data.append(image/256)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image/256)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])


In [14]:
training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64).shuffle(640).repeat(100)

C:\Users\carlos\AppData\Local\Temp/ipykernel_23244/1242183787.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(labels)


In [24]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21744/3660101917.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)


In [15]:
real_dataset

<BatchDataset shapes: ((None, 24, 94, 3), (None, 12)), types: (tf.float32, tf.int32)>